# Asymmetry + DEAP (emotion)

In this tutorial, we will be working on the DEAP dataset, a benchmark EEG emotion recognition dataset. 

In this dataset, there is a total of 32 participants, where each participant watches 40 1-minute videos.  Thus <code>s01.dat</code> is holding 40 batches.   For simplicity, we shall only use the first participant, i.e., <code>s01.dat</code>.   

You can download the dataset by googling and ask permission from the owner by filling a form.

Looking in each dat file (e.g., s01), it contains the data and label
- Data ----- 40 x 40 x 8064 [	video/batches x channel x samples ]
- Label  ---- 40 x 4 

Out of 40 channels, 32 channels were of EEG, and the rest of 8 of them from other sensors such as EOG (see the section 6.1 of the original paper).  We shall only extract the first 32 channels.   For the 8064, since the data is downsampled to 128Hz, thus one second contains around 128 samples, thus in one minute which is 60 seconds, it will be roughly 7680 samples.  The paper did not really talk a lot but it is likely there is  another 1.5 seconds before and after which total to 8064 samples (128 Hz * 63 seconds).

The four labels correspond to valence, arousal, liking, and dominance, in this order.  We will only use valence and arousal, thus index 0 and 1 of the labels will be extracted.

In [12]:
import pickle
import os

import mne
from mne import create_info
from mne.io import RawArray

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [13]:
np.set_printoptions(precision=4)

In [14]:
#this will set the plotting size
mne.set_config('MNE_BROWSE_RAW_SIZE','10,5')  

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [16]:
# torch.cuda.get_device_name(0)

## 1. Loading the dataset

Let's first create a simple dataset loader.   The code is explained using comments and is quite self-explanatory.

In [17]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path, stim):
        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')
            all_data.append(temp['data'])
            
            if stim == "Valence":
                all_label.append(temp['labels'][:,:1])   #the first index is valence
            elif stim == "Arousal":
                all_label.append(temp['labels'][:,1:2]) # Arousal  #the second index is arousal
                
        self.data = np.vstack(all_data)   #shape: (40, 40, 8064) ==> data of 1 participant
        self.label = np.vstack(all_label) #(40, )  ==> 1280 samples, each with a unique label (depend on the param "stim")
        
        del temp, all_data, all_label

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        single_data  = self.data[idx]
        single_label = (self.label[idx] > 5).astype(float)   #convert the scale to either 0 or 1 (to classification problem)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }
        
        return batch

Let's try load the dataset.

In [18]:
path = "data/deap"  #create a folder "data", and inside put s01.dat,....,s32.dat inside from the preprocessed folder from the DEAP dataset

In [19]:
dataset_valence = Dataset(path, "Valence")
dataset_arousal = Dataset(path, "Arousal")

We can try look at one sample using the index.  This is automatically mapped to the <code>__getitem__</code> function in the <code>Dataset</code> class.

In [20]:
dataset_valence[0]

{'data': tensor([[ 9.4823e-01,  1.6533e+00,  3.0137e+00,  ..., -2.8265e+00,
          -4.4772e+00, -3.6769e+00],
         [ 1.2471e-01,  1.3901e+00,  1.8351e+00,  ..., -2.9870e+00,
          -6.2878e+00, -4.4743e+00],
         [-2.2165e+00,  2.2920e+00,  2.7464e+00,  ..., -2.6371e+00,
          -7.4065e+00, -6.7559e+00],
         ...,
         [ 2.3078e+02,  6.9672e+02,  1.1951e+03,  ...,  1.0108e+03,
           1.2831e+03,  1.5200e+03],
         [-1.5418e+03, -1.6180e+03, -1.6927e+03,  ..., -1.5784e+04,
          -1.5782e+04, -1.5781e+04],
         [ 6.3905e-03,  6.3905e-03,  6.3905e-03,  ..., -9.7608e-02,
          -9.7608e-02, -9.7608e-02]]),
 'label': tensor([1.])}

In [21]:
print("Shape of data: ", dataset_valence[0]['data'].shape)  #40 channels of data, 8064 samples in 1 minute
print("Shape of label: ", dataset_valence[0]['label'].shape) #just 1 single label; 0 or 1

Shape of data:  torch.Size([40, 8064])
Shape of label:  torch.Size([1])


Let's try to look at our data and label distribution.

In [22]:
data = dataset_valence[:]['data']
label = dataset_valence[:]['label']

In [23]:
#so we got 40 trials (40 videos, each with 40 channels of data, each video contains 8064 EEG samples)
data.shape  

torch.Size([40, 40, 8064])

In [24]:
#so we got 40 labels, i.e., one label per video
label.shape  

torch.Size([40, 1])

Let's count how many 0 and 1 in the valence dataset, to see if there is some imbalance.

In [25]:
cond_1 = label == 1
cond_0 = label == 0

print("Labels 1 in valence dataset: ", len(label[cond_1]))
print("Labels 0 in valence dataset: ", len(label[cond_0]))

Labels 1 in valence dataset:  19
Labels 0 in valence dataset:  21


Let's also count in the arousal dataset, to see if there is some imbalance.

In [26]:
cond_1 = label == 1
cond_0 = label == 0

print("Labels 1 in arousal dataset: ", len(label[cond_1]))
print("Labels 0 in arousal dataset: ", len(label[cond_0]))

Labels 1 in arousal dataset:  19
Labels 0 in arousal dataset:  21


To confirm that the first 32 channels are EEG and the rest of the 8 channels are other channels, let's check the median value of each channel to see whether there is a pattern.

In [29]:
for i in range(40):
    print(f"Median of {i} data: {torch.median(data[:, i, :]):.4f}")

Median of 0 data: 0.0193
Median of 1 data: 0.0043
Median of 2 data: 0.0012
Median of 3 data: 0.0000
Median of 4 data: 0.0205
Median of 5 data: 0.0323
Median of 6 data: -0.0175
Median of 7 data: 0.0104
Median of 8 data: -0.0291
Median of 9 data: -0.0003
Median of 10 data: 0.0255
Median of 11 data: -0.0350
Median of 12 data: -0.0393
Median of 13 data: -0.0604
Median of 14 data: -0.0421
Median of 15 data: -0.0049
Median of 16 data: -0.0088
Median of 17 data: 0.0713
Median of 18 data: 0.0021
Median of 19 data: 0.0022
Median of 20 data: 0.0238
Median of 21 data: 0.0390
Median of 22 data: 0.0121
Median of 23 data: 0.0137
Median of 24 data: 0.0462
Median of 25 data: 0.0395
Median of 26 data: 0.0184
Median of 27 data: 0.0176
Median of 28 data: 0.0353
Median of 29 data: -0.0102
Median of 30 data: 0.0165
Median of 31 data: -0.0148
Median of 32 data: -25.4018
Median of 33 data: 43.8558
Median of 34 data: -3.7539
Median of 35 data: 8.1745
Median of 36 data: 6808.3848
Median of 37 data: 48.6546
Med

As we can see, the data index 0 to 31 is clearly EEG, while data from 32 onward is not.

## 2. Artifact Removal

Note that since the data is already preprocessed by the authors, we don't have to do anything more, but it's very natural for us to do preprocessing, e.g., min-max normalization, notch filters, band pass filters, etc.

## 3. EDA - Spectral Analysis

In this part 2, we will focus on feature engineering using spectral analysis.  Spectral analysis here refers to the analysis of theta (4 - 8 Hz), alpha (8 - 12 Hz), beta (12 - 30 Hz), and gamma (30 - 64 Hz).   

Spectral analysis is a very basic and must-do analysis for emotions/cognitions/resting state since it is a common knowledge with abundant evidence that our emotion/cognition change how our brain signals oscillate.  For example, when we are calm, alpha is relatively high, likewise, when we are attentive, beta is relatively high and alpha becomes relatively lower.

In this part, we shall extract these powers as features and then input them into SVM and see if these features are useful for predicting the four valence-arousal classes that we have obtained from Part 1.